In [7]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.fft import fft
import pywt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
file_path = 'Dataset/dataset.csv'
data = pd.read_csv(file_path)

# Function to extract statistical features
def extract_statistical_features(row):
    features = []
    for col in ['X', 'Y', 'Z', 'Mixed']:
        features.append(row[col].mean())
        features.append(row[col].std())
        features.append(skew(row[col]))
        features.append(kurtosis(row[col]))
    return features

# Function to extract FFT features
def extract_fft_features(row):
    features = []
    for col in ['X', 'Y', 'Z', 'Mixed']:
        fft_values = np.abs(fft(row[col].values)[:10])
        features.extend(fft_values)
    return features

# Function to extract DWT features
def extract_dwt_features(row):
    features = []
    for col in ['X', 'Y', 'Z', 'Mixed']:
        coeffs = pywt.wavedec(row[col].values, 'db1', level=2)
        features.extend(coeffs[0])  # Approximation coefficients
    return features

# Extract features for each row
feature_list = []
for i in range(len(data)):
    row = data.iloc[i:i+1]
    features = []
    features.extend(extract_statistical_features(row))
    features.extend(extract_fft_features(row))
    features.extend(extract_dwt_features(row))
    feature_list.append(features)

# Create a new DataFrame with extracted features
feature_df = pd.DataFrame(feature_list)
feature_df['ClassLabel'] = data['ClassLabel'].values

# Check for any NaN values and handle them
feature_df.fillna(0, inplace=True)

# Separate features and labels
X = feature_df.drop('ClassLabel', axis=1)
y = feature_df['ClassLabel']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the SVM model
svm_model = SVC(kernel='linear')

# Train the SVM model
svm_model.fit(X_train, y_train)

# Make predictions
y_pred = svm_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


/home/dasun/.local/lib/python3.10/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 2 is too high: all coefficients will experience boundary effects.
  warnings.warn(


Accuracy: 0.42157511581733953
Classification Report:
              precision    recall  f1-score   support

           1       0.27      0.25      0.26       310
           2       0.67      0.90      0.77       319
           3       0.41      0.33      0.36       282
           4       0.32      0.53      0.40       303
           5       0.29      0.06      0.10       297

    accuracy                           0.42      1511
   macro avg       0.39      0.41      0.38      1511
weighted avg       0.39      0.42      0.38      1511



In [8]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.fft import fft
import pywt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Load the dataset
file_path = 'Dataset/dataset.csv'
data = pd.read_csv(file_path)

# Function to extract statistical features
def extract_statistical_features(row):
    features = []
    for col in ['X', 'Y', 'Z', 'Mixed']:
        features.append(row[col].mean())
        features.append(row[col].std())
        features.append(skew(row[col]))
        features.append(kurtosis(row[col]))
    return features

# Function to extract FFT features
def extract_fft_features(row):
    features = []
    for col in ['X', 'Y', 'Z', 'Mixed']:
        fft_values = np.abs(fft(row[col].values)[:10])
        features.extend(fft_values)
    return features

# Function to extract DWT features
def extract_dwt_features(row):
    features = []
    for col in ['X', 'Y', 'Z', 'Mixed']:
        coeffs = pywt.wavedec(row[col].values, 'db1', level=2)
        features.extend(coeffs[0])  # Approximation coefficients
    return features

# Extract features for each row
feature_list = []
for i in range(len(data)):
    row = data.iloc[i:i+1]
    features = []
    features.extend(extract_statistical_features(row))
    features.extend(extract_fft_features(row))
    features.extend(extract_dwt_features(row))
    feature_list.append(features)

# Create a new DataFrame with extracted features
feature_df = pd.DataFrame(feature_list)
feature_df['ClassLabel'] = data['ClassLabel'].values

# Check for any NaN values and handle them
feature_df.fillna(0, inplace=True)

# Separate features and labels
X = feature_df.drop('ClassLabel', axis=1)
y = feature_df['ClassLabel']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Initialize the SVM model
svm_model = SVC()

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(X_train, y_train)

# Make predictions
y_pred = grid.best_estimator_.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Best Parameters: {grid.best_params_}")
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


/home/dasun/.local/lib/python3.10/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 2 is too high: all coefficients will experience boundary effects.
  warnings.warn(


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.3s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.3s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.3s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.3s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.3s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   0.3s
[CV] END ......................C=0.1, gamma=0.01

In [10]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.fft import fft
import pywt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Load the dataset
file_path = 'Dataset/dataset.csv'
data = pd.read_csv(file_path)

# Function to extract statistical features
def extract_statistical_features(row):
    features = []
    for col in ['X', 'Y', 'Z', 'Mixed']:
        features.append(row[col].mean())
        features.append(row[col].std())
        features.append(skew(row[col]))
        features.append(kurtosis(row[col]))
    return features

# Function to extract FFT features
def extract_fft_features(row):
    features = []
    for col in ['X', 'Y', 'Z', 'Mixed']:
        fft_values = np.abs(fft(row[col].values)[:10])
        features.extend(fft_values)
    return features

# Function to extract DWT features
def extract_dwt_features(row):
    features = []
    for col in ['X', 'Y', 'Z', 'Mixed']:
        coeffs = pywt.wavedec(row[col].values, 'db1', level=2)
        features.extend(coeffs[0])  # Approximation coefficients
    return features

# Extract features for each row
feature_list = []
for i in range(len(data)):
    row = data.iloc[i:i+1]
    features = []
    features.extend(extract_statistical_features(row))
    features.extend(extract_fft_features(row))
    features.extend(extract_dwt_features(row))
    feature_list.append(features)

# Create a new DataFrame with extracted features
feature_df = pd.DataFrame(feature_list)
feature_df['ClassLabel'] = data['ClassLabel'].values

# Check for any NaN values and handle them
feature_df.fillna(0, inplace=True)

# Separate features and labels
X = feature_df.drop('ClassLabel', axis=1)
y = feature_df['ClassLabel']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Initialize the SVM model with class weights
svm_model = SVC(class_weight='balanced')

# Hyperparameter tuning using GridSearchCV with expanded search space
param_grid = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf', 'poly', 'sigmoid']
}

grid = GridSearchCV(SVC(class_weight='balanced'), param_grid, refit=True, verbose=2)
grid.fit(X_train, y_train)

# Make predictions
y_pred = grid.best_estimator_.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Best Parameters: {grid.best_params_}")
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)



/home/dasun/.local/lib/python3.10/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 2 is too high: all coefficients will experience boundary effects.
  warnings.warn(


Fitting 5 folds for each of 75 candidates, totalling 375 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.4s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   0.5s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   0.9s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   0.8s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   0.9s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   0.7s
[CV] END .....................C=0.1, gamma=1, kernel=sigmoid; total time=   0.4s
[CV] END .....................C=0.1, gamma=1, k